<a href="https://colab.research.google.com/github/NgThVinh/ChatLM-Playground/blob/main/ChatModel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Install Lib

In [1]:
!pip install -qU transformers trl accelerate bitsandbytes

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.0/44.0 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.0/10.0 MB 110.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.9/318.9 kB 23.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 342.1/342.1 kB 27.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.1/76.1 MB 28.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 485.4/485.4 kB 21.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 95.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 79.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 51.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

# Model Finetuning

## Without pipeline

**List models:**
*   microsoft/Phi-4-mini-instruct
*   Qwen/Qwen2.5-7B
*   deepseek-ai/DeepSeek-R1-Distill-Llama-8B
*   mistralai/Mistral-7B-Instruct-v0.3
*   google/gemma-2-9b



In [2]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig

In [95]:
checkpoint = "NousResearch/Hermes-2-Pro-Llama-3-8B"
quantization_config = BitsAndBytesConfig(
    load_in_8bit=True
)
tokenizer = AutoTokenizer.from_pretrained(checkpoint)
model = AutoModelForCausalLM.from_pretrained(checkpoint,
                                             torch_dtype=torch.float16,
                                             device_map="auto",
                                             quantization_config=quantization_config
                                             )

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [94]:
import torch
import gc
# del model
torch.cuda.empty_cache()
gc.collect()

9

In [96]:
tokenizer.vocab_size

128000

In [97]:
chat = [
    { "role": "user", "content": "Hi, who are you?"},
]
question = tokenizer.apply_chat_template(chat,
                                         tokenize=False,
                                         add_generation_prompt=True
                                         )
print(question)

<|begin_of_text|><|im_start|>user
Hi, who are you?<|im_end|>
<|im_start|>assistant



In [98]:
def get_current_temperature(location: str, unit: str) -> float:
    """
    Get the current temperature at a location.

    Args:
        location: The location to get the temperature for, in the format "City, Country"
        unit: The unit to return the temperature in. (choices: ["celsius", "fahrenheit"])
    Returns:
        The current temperature at the specified location in the specified units, as a float.
    """
    return 22.  # A real function should probably actually get the temperature!

def get_current_wind_speed(location: str) -> float:
    """
    Get the current wind speed in km/h at a given location.

    Args:
        location: The location to get the temperature for, in the format "City, Country"
    Returns:
        The current wind speed at the given location in km/h, as a float.
    """
    return 6.  # A real function should probably actually get the wind speed!

tools = [get_current_temperature, get_current_wind_speed]

In [100]:
messages = [
  {"role": "system", "content": "You are a bot that responds to weather queries. You should reply with the unit used in the queried location."},
  {"role": "user", "content": "Hey, what's the temperature in Paris right now?"}
]
inputs = tokenizer.apply_chat_template(messages,
                                       tokenize=False,
                                       add_generation_prompt=True,
                                       tools=tools,
                                       )
print(inputs)

<|begin_of_text|><|im_start|>system
You are a function calling AI model. You are provided with function signatures within <tools></tools> XML tags. You may call one or more functions to assist with the user query. Don't make assumptions about what values to plug into functions. Here are the available tools: <tools> {"type": "function", "function": {"name": "get_current_temperature", "description": "get_current_temperature(location: str, unit: str) -> float - Get the current temperature at a location.

    Args:
        location(str): The location to get the temperature for, in the format "City, Country"        unit(str): The unit to return the temperature in.
    Returns:
        The current temperature at the specified location in the specified units, as a float.", "parameters": {"type": "object", "properties": {"location": {"type": "string", "description": "The location to get the temperature for, in the format \"City, Country\""}, "unit": {"type": "string", "enum": ["celsius", "fahr

In [101]:
inputs = tokenizer.apply_chat_template(messages,
                                       add_generation_prompt=True,
                                       return_dict=True,
                                       tools=tools,
                                       return_tensors="pt"
                                       ).to("cuda")

In [102]:
from transformers import TextStreamer

streamer = TextStreamer(tokenizer)

In [103]:
outputs = model.generate(**inputs,
                         streamer=streamer,
                         pad_token_id=tokenizer.eos_token_id,
                         max_new_tokens=1024
                         )
# outputs

Setting `pad_token_id` to `eos_token_id`:128003 for open-end generation.


<|begin_of_text|><|im_start|>system
You are a function calling AI model. You are provided with function signatures within <tools></tools> XML tags. You may call one or more functions to assist with the user query. Don't make assumptions about what values to plug into functions. Here are the available tools: <tools> {"type": "function", "function": {"name": "get_current_temperature", "description": "get_current_temperature(location: str, unit: str) -> float - Get the current temperature at a location.

    Args:
        location(str): The location to get the temperature for, in the format "City, Country"        unit(str): The unit to return the temperature in.
    Returns:
        The current temperature at the specified location in the specified units, as a float.", "parameters": {"type": "object", "properties": {"location": {"type": "string", "description": "The location to get the temperature for, in the format \"City, Country\""}, "unit": {"type": "string", "enum": ["celsius", "fahr

In [104]:
print(tokenizer.decode(outputs[0][len(inputs["input_ids"][0]):]))

<tool_call>
{"name": "get_current_temperature", "arguments": {"location": "Paris, France", "unit": "celsius"}}
</tool_call><|im_end|>


In [ ]:

# streamer = TextStreamer(tokenizer)
# outputs = model.generate(**inputs,  max_new_tokens=1000)

## With pipeline

In [ ]:
pipe = pipeline("text-generation", model=model, tokenizer=tokenizer)

Device set to use cuda


In [ ]:
messages = [
    {"role": "user", "content": "Who are you?"},
]
pipe(messages)

[{'generated_text': [{'role': 'user', 'content': 'Who are you?'},
   {'role': 'assistant',
    'content': "Greetings! I'm DeepSeek-R1, an artificial intelligence assistant created by DeepSeek. I'm"}]}]

In [ ]:
messages=[]
while True:
  user_input = input("User: ")
  if user_input.lower() == "q":
    break
  messages.append({"role": "user", "content": user_input})
  response = pipe(messages)
  messages = response[0]['generated_text']
  print("Assistant:", response[0]['generated_text'][-1]['content'])

User: When I say any thing, response with the first and the last word I said
Assistant: Okay, so I'm trying to figure out how to respond to the user's request. They said, "When I say any thing, response with the first and the last word I said." Hmm, that's a bit confusing. Let me break it down.

First, the user wants a response that includes the first and the last word of whatever they say. So, for example, if they say, "Hello, how are you?" the response should be "Hello, how are you." Wait, that's just the same as what they said. Maybe I'm misunderstanding.

Let me think about it again. Maybe they mean that whenever they say something, I should respond by taking the first word and the last word of their message and put them together, perhaps with some action or another word in the middle. Like, if they say "I am happy," the response could be "I am happy." But that seems redundant.

Wait, perhaps they want the first and last words of their message to be part of my response, but not nec

KeyboardInterrupt: Interrupted by user

In [ ]:
# train_grpo.py
from datasets import load_dataset
from trl import GRPOConfig, GRPOTrainer

dataset = load_dataset("trl-lib/tldr", split="train")

# Define the reward function, which rewards completions that are close to 20 characters
def reward_len(completions, **kwargs):
    return [-abs(20 - len(completion)) for completion in completions]

training_args = GRPOConfig(output_dir="content/Qwen2-0.5B-GRPO", logging_steps=10)
trainer = GRPOTrainer(
    model="Qwen/Qwen2-0.5B-Instruct",
    reward_funcs=reward_len,
    args=training_args,
    train_dataset=dataset,
)
trainer.train()

Sliding Window Attention is enabled but not implemented for `sdpa`; unexpected results may be encountered.
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: ngthevinh06 (ngthvinh) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Step,Training Loss


OutOfMemoryError: CUDA out of memory. Tried to allocate 150.00 MiB. GPU 0 has a total capacity of 22.16 GiB of which 21.38 MiB is free. Process 694410 has 22.13 GiB memory in use. Of the allocated memory 20.33 GiB is allocated by PyTorch, and 1.58 GiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)